<a href="https://colab.research.google.com/github/yurikaminski/Saga/blob/master/Text_summarization_using_NLTK_and_Pandas_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sumarização de textos com NLTK (Natural Language Toolkit) e Pandas em Português

##1) Preparando as bibliotecas
Utilizaremos as bibliotecas: 
1.  **urllib** para abrir o site alvo.
2. **BeautifulSoup** para scanear o site e localizar o conteúdo que queremos sumarizar.
3. **NLTK/tokenize** para tokenizar as palavras. 
4.  **NLTK/stopwords** para descartar palavras de uso comum - Stopwords (de, e, mas, para, a...).
5. **NLTK/probability** para encontrar a frequência de aparição das palavras no texto.
6. **Pandas** para manipular nossas listas de palavras e sentenças
6. **String/punctuation** para descartar também as pontuações (Que são tokenizados também como palavras)
7. **Collections/Defaultdict** para separar as sentenças mais importantes.

In [93]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import pandas as pd
from heapq import nlargest
from string import punctuation
from collections import defaultdict



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##2) Acessamos e "bufferizamos" o site

In [0]:
link = Request('http://ultimosegundo.ig.com.br/politica/2017-04-25/reforma-da-previdencia.html',
               headers={'User-Agent': 'Mozilla/5.0'})
pagina = urlopen(link).read().decode('utf-8', 'ignore')
#print(pagina)

##3) Extraimos a notícia que nos interessa usando a biblioteca BeautifulSoup. 
O código depende da estrutura da página que estamos garimpando, ou seja, é preciso modificá-lo para garimpar outras páginas, ou criar alguma estrutura de regras para tentarmos "adivinhar" em que parte do HTML está o conteúdo do site. Nesse caso, a notícia se encontra em uma parte com div id="noticia". Observe que existe uma imagem dentro dessa seção, podemos excluí-la também

In [95]:
soup = BeautifulSoup(pagina, "lxml")
#soup.figcaption.decompose(id="img alt")

noticia = soup.find(id="noticia")
#print(noticia)
#Vamos eliminar a imagem e sua legenda usando a função decompose
noticia.figure.decompose()
#print(noticia)
text = noticia.text
print(text)



A comissão especial que analisa a proposta de reforma da Previdência na Câmara dos Deputados inicia na tarde desta terça-feira (25) a discussão do                   relatório apresentado na semana passada pelo relator
, deputado Arthur Maia (PPS-BA).
Depois de fechar acordo com parlamentares da oposição, que tentavam obstruir a sessão de leitura do parecer do relator, o presidente da comissão da                   reforma da Previdência
, deputado Carlos Marun (PMDB-MS), designou que todas as reuniões desta semana sejam para discutir o relatório e apresentar pedido de vista.
O acordo com a oposição ainda definiu que a votação do relatório pelos membros da comissão deve ocorrer na próxima semana, dia 2 de maio. Já a partir do dia 8, o relatório estaria pronto para ser votado no plenário da Câmara dos Deputados. Para que isso aconteça, a equipe do governo Temer segue atuando para conquistar os votos necessários para aprovar as mudanças nas regras para a aposentadoria.
Para reduzir a res

##4)  Vamos dividir o nosso texto em sentenças e em seguida em palavras  usando NLTK tokenization
A tokenização consistem em separar as palavras ou sentenças em vetores de strings.

In [163]:
sentences = sent_tokenize(text)
print(sentences)
words = word_tokenize(text)
#print(words)

['\n\nA comissão especial que analisa a proposta de reforma da Previdência na Câmara dos Deputados inicia na tarde desta terça-feira (25) a discussão do                   relatório apresentado na semana passada pelo relator\n, deputado Arthur Maia (PPS-BA).', 'Depois de fechar acordo com parlamentares da oposição, que tentavam obstruir a sessão de leitura do parecer do relator, o presidente da comissão da                   reforma da Previdência\n, deputado Carlos Marun (PMDB-MS), designou que todas as reuniões desta semana sejam para discutir o relatório e apresentar pedido de vista.', 'O acordo com a oposição ainda definiu que a votação do relatório pelos membros da comissão deve ocorrer na próxima semana, dia 2 de maio.', 'Já a partir do dia 8, o relatório estaria pronto para ser votado no plenário da Câmara dos Deputados.', 'Para que isso aconteça, a equipe do governo Temer\xa0segue atuando para conquistar os votos necessários para aprovar as mudanças nas regras para a aposentadori

##5) Agora removemos as stopwords da nossa lista. Por serem muito frequentes, elas podem deturpar a análise que relaciona a frequencia à relevância.

In [97]:
complete_stopwords = set(stopwords.words('portuguese') + list(punctuation))
clean_words = [word for word in words if word not in complete_stopwords]
print(clean_words[1:5])

['comissão', 'especial', 'analisa', 'proposta']


##6) Vamos analisar a distribuição de frequência das palavras
A distribuição de frequência é a quantidade de vezes que uma palavra aparece no texto pelo número de palavras totais no texto. Além disso, vamos colocar essa lista em um Datadframe de pandas para fazermos algumas análises interessantes de maneira mais ágil.

In [173]:
frequency = FreqDist(clean_words)
#Colocamos a estrutura em um dataframe para obtermos algumas estatísticas básicas
frequency_pd = pd.DataFrame(list(frequency.items()), columns = ["Word","Frequency"])
frequency_pd.sort_values(by='Frequency', inplace=True, ascending=False)
frequency_pd.head() #Displays the first 5 lines of the table

,Word,Frequency
98,anos,10
1,comissão,7
45,O,6
15,relatório,6
5,reforma,6


In [99]:
frequency_pd.describe()#Displays basic stats of the distribution

,Frequency
count,247.000000
mean,1.538462
std,1.258119
min,1.000000
25%,1.000000
50%,1.000000
75%,1.500000
max,10.000000


##7) Criamos um escore para as sentenças, somando as frequências das palavras que aparecem nelas.
Quanto mais palavras importantes aparecem na sentença, mais ela será considerada importante. No sumário, nós usaremos as N sentenças mais importantes do texto.

In [137]:
#Criamos um dataframe para colocar as sentenças e seus respectivos escores
sentences_df = pd.DataFrame(columns=['Sentences','Orig_Index','Score'])
sentences_df['Sentences'] = sentences
sentences_df['Orig_Index'] = sentences_df.index
sentences_df['Score'] = 0
sentences_df.head()

,Sentences,Orig_Index,Score
0,\n\nA comissão especial que analisa a proposta...,0,0
1,Depois de fechar acordo com parlamentares da o...,1,0
2,O acordo com a oposição ainda definiu que a vo...,2,0
3,"Já a partir do dia 8, o relatório estaria pron...",3,0
4,"Para que isso aconteça, a equipe do governo Te...",4,0


In [164]:
#Em seguida populamos a coluna escore com a soma da frequencia de cada palavra que compõe a frase:

for i,sentence in enumerate(sentences):
    score = 0
    for word in word_tokenize(sentence):
      if word in frequency:
        score += frequency[word]
    sentences_df.loc[i,'Score']=score
    #print(i,score)

sentences_df.head()

,Sentences,Orig_Index,Score
26,Leia também: Lava Jato: STJ recebe pedido de i...,26,24
25,"Caso os senadores modifiquem o texto, o projet...",25,12
24,"Para se tornar lei, a proposta de reforma da P...",24,67
23,O relatório ainda pode ser alterado durante as...,23,27
22,"Já na próxima semana, após o feriado, votar a ...",22,57


##8) Agora ordenamos pelo escore e escolhemos os 10 maiores pesos.
Cortamos o dataframe com os 10 maiores pesos e reordenamos baseados no índice original para que as sentenças sejam escritas na ordem em que aparecem no texto original.

In [172]:
sentences_df.sort_values(by='Score',inplace = True, ascending=False)
#sentences_df.head()
top = 10
important_sentences_df = pd.DataFrame(sentences_df[0:top])
important_sentences_df.sort_values(by='Orig_Index',inplace = True, ascending=True)
#important_sentences_df.head()
print(important_sentences_df['Sentences'])

0     \n\nA comissão especial que analisa a proposta...
1     Depois de fechar acordo com parlamentares da o...
2     O acordo com a oposição ainda definiu que a vo...
6     O relatório de Arthur Maia fixa a idade mínima...
8     Para a aposentadoria por tempo de contribuição...
9     Aí é só checar na tabela do aumento progressiv...
12    Leia também: Após assalto milionário no Paragu...
19    O líder reiterou que o plenário deve votar a r...
22    Já na próxima semana, após o feriado, votar a ...
24    Para se tornar lei, a proposta de reforma da P...
Name: Sentences, dtype: object
